# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [1]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 15:50:56--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.08s  

2016-10-24 15:50:56 (57.4 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

In [2]:
!mv abq-food-inspections-fy2010.csv FoodInspection.csv

In [3]:
!csvcut -n FoodInspection.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


Here, we find 27 unique columns in the dataset. This data set primarily deals with respective restaurants in the city and their inspection type and results.

In [4]:
!csvcut -c1,2,3,4,5,6,7 FoodInspection.csv | head -3| csvlook

|----------------------+--------------+-------------------------------+-------------+-------+-------+------------|
|  FACILITY_NAME       | FACILITY_KEY | SITE_ADDRESS                  | CITY        | STATE | ZIP   | OWNER_KEY  |
|----------------------+--------------+-------------------------------+-------------+-------+-------+------------|
|  00BSAY IT WITH CAKE | 50534        | 10200 GOLF COURSE RD NW STE B | ALBUQUERQUE | NM    | 87114 | 47341      |
|  00BSAY IT WITH CAKE | 50534        | 10200 GOLF COURSE RD NW STE B | ALBUQUERQUE | NM    | 87114 | 47341      |
|----------------------+--------------+-------------------------------+-------------+-------+-------+------------|


In [5]:
!csvcut -c8,9,10,11,12,13 FoodInspection.csv | head -3| csvlook

|----------------------+--------------------+---------------+-------------+-------------+-------------------|
|  OWNER_NAME          | NATURE_OF_BUSINESS | STREET_NUMBER | STREET_NAME | STREET_TYPE | POST_DIRECTIONAL  |
|----------------------+--------------------+---------------+-------------+-------------+-------------------|
|  OOBSAY IT WITH CAKE | CAKE DECORATING    | 10200         | GOLF COURSE | RD          | NW                |
|  OOBSAY IT WITH CAKE | CAKE DECORATING    | 10200         | GOLF COURSE | RD          | NW                |
|----------------------+--------------------+---------------+-------------+-------------+-------------------|


The First 13 columns deal with the restaurant and corresponding addresses, along with two sets of keys. 

In [6]:
!csvcut -c14,15,16,17,18 FoodInspection.csv | head -3| csvlook

|-------------+------------------+------------------------------+---------------------+------------------|
|  PHONE      | PROGRAM_CATEGORY | PROGRAM_CATEGORY_DESCRIPTION | INSPECTION_DATE     | INSPECTION_TYPE  |
|-------------+------------------+------------------------------+---------------------+------------------|
|  5058339563 | 0405             | Food Processor               | 2009-10-21 00:00:00 |                  |
|  5058339563 | 0405             | Food Processor               | 2009-10-21 00:00:00 |                  |
|-------------+------------------+------------------------------+---------------------+------------------|


In [7]:
!csvcut -c19,20,21,22,23,24,25 FoodInspection.csv | head -3| csvlook

|--------------------------+------------+-------------+-------------+-------------+----------------+-----------------|
|  INSPECTION_DESC         | SERIAL_NUM | ACTION_CODE | ACTION_DESC | RESULT_CODE | RESULT_DESC    | VIOLATION_CODE  |
|--------------------------+------------+-------------+-------------+-------------+----------------+-----------------|
|  EMERGENCY RESPONSE      | DA8009125  | 00          | NON-GRADED  | 00          | NOT APPLICABLE |     68          |
|  ROUTINE FOOD INSPECTION | DA8009123  | 11          | APPROVED    | 03          | IN COMPLIANCE  |     00          |
|--------------------------+------------+-------------+-------------+-------------+----------------+-----------------|


In [8]:
!csvcut -c26,27 FoodInspection.csv | head -3| csvlook

|-----------------------------------+------------------|
|  VIOLATION_DESC                   | INSPECTION_MEMO  |
|-----------------------------------+------------------|
|  Delivered Informational Brochure |                  |
|  No Violations Found              |                  |
|-----------------------------------+------------------|


The remaining columns deal with the inspection of each restaurant, the type of the inspection and the results. Here, we see a bunch of codes. The codes are internal to the inspection metrics and we cannot infer what they mean. But looks there may be a few null values; few empty spots in the Inspection_Memo column. So let's explore that.

In [9]:
!csvcut -c2,6,7,10,14,15,21,23,25 FoodInspection.csv | csvstat

  1. FACILITY_KEY
	<class 'int'>
	Nulls: False
	Min: 13
	Max: 101482
	Sum: 820405773
	Mean: 46730.79135338346
	Median: 50866.0
	Standard Deviation: 32659.794578837962
	Unique values: 2835
	5 most frequent values:
		46323:	93
		65863:	85
		32817:	68
		80949:	68
		91825:	66
  2. ZIP
	<class 'str'>
	Nulls: False
	Unique values: 48
	5 most frequent values:
		87110:	1996
		87108:	1976
		87102:	1601
		87106:	1568
		87109:	1552
	Max length: 10
  3. OWNER_KEY
	<class 'int'>
	Nulls: False
	Min: 11
	Max: 87302
	Sum: 644760848
	Mean: 36725.953975848715
	Median: 33747.0
	Standard Deviation: 29428.207216739836
	Unique values: 1892
	5 most frequent values:
		1838:	792
		3537:	321
		1272:	228
		36280:	156
		80274:	155
  4. STREET_NUMBER
	<class 'str'>
	Nulls: True
	Unique values: 1349
	5 most frequent values:
		6600:	216
		10000:	152
		300:	143
		2200:	127
		8100:	125
	Max length: 7
  5. PHONE
	<class 'int'>
	Nulls: True
	Min: 2471094
	Max: 505345566137310
	Sum: 2608539289633190
	Mean: 161719732773.2

In [10]:
!csvcut -c1,3,4,5,8,9,11,12,13,16,18,19,20,22,24,26,27 FoodInspection.csv | csvstat

  1. FACILITY_NAME
	<class 'str'>
	Nulls: False
	Unique values: 2615
	5 most frequent values:
		SMITHS FOOD AND DRUG CENTERS INC:	154
		ISOTOPES PARK - OVATIONS:	93
		PER EHSII CECELIA GARCIA OOB LA REYNA DEL SUR:	85
		ECHO INC:	68
		ISLETA AMPHITHEATER:	68
	Max length: 77
  2. SITE_ADDRESS
	<class 'str'>
	Nulls: True
	Unique values: 2456
	5 most frequent values:
		1601 AVENIDA CESAR CHAVEZ SE:	94
		644 OLD COORS BLVD SW:	85
		8601 CENTRAL AV NE:	77
		2200 SUNPORT BLVD SE:	75
		5210 CENTRAL AV SE:	73
	Max length: 35
  3. CITY
	<class 'str'>
	Nulls: False
	Values: ALBUQUERQUE
  4. STATE
	<class 'str'>
	Nulls: False
	Values: 87, NM
  5. OWNER_NAME
	<class 'str'>
	Nulls: False
	Unique values: 1852
	5 most frequent values:
		APS:	792
		SMITHS FOOD AND DRUG CENTERS INC:	321
		CIRCLE K STORES INC:	277
		ALBERTSONS:	228
		WAL MART STORES EAST  LP:	156
	Max length: 82
  6. NATURE_OF_BUSINESS
	<class 'str'>
	Nulls: True
	Unique values: 594
	5 most frequent values:
		RESTAURANT:	4994
		FOOD SERV

In [11]:
!csvcut -c17 FoodInspection.csv | csvstat

  1. INSPECTION_DATE
	<class 'datetime.date'>
	Nulls: False
	Min: 2009-07-01
	Max: 2010-06-30
	Unique values: 258
	5 most frequent values:
		2009-12-16:	184
		2009-09-10:	159
		2009-09-25:	148
		2009-11-04:	133
		2009-10-01:	131

Row count: 17556


Well, as it turns out quite few columns have null values, specifically columns 3,8,9,10,11,12,13,14,16,24,25,26. From missing restaurant addresses to contact information and results of the inspecton. This could result in an incomplete database. 

I am most interested in finding out if each unique restaurant has an associated inspection decision and violation decision. Additionlly, if there are any corresponding notes or actions. The whole purpose is to make sure each restaurant in the database is evaluated; so we need to ensure that. 

I would like to explore if there is a particular chain / group of restaurants which have non-compliance results. Perhaprs a particular set of owners are irresponsible or they are in congregated in a particular area in the city, etc. 

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

In [12]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

--2016-10-24 15:51:03--  https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.32.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.32.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5025226 (4.8M) [text/plain]
Saving to: ‘abq-food-inspections-fy2010.csv’

abq-food-inspection 100%[=====================>]   4.79M  --.-KB/s   in 0.09s  

2016-10-24 15:51:04 (56.0 MB/s) - ‘abq-food-inspections-fy2010.csv’ saved [5025226/5025226]



In [13]:
cp abq-food-inspections-fy2010.csv inspection2010.csv

Here, I have to change the name of the file because for some reason i keep getting errors using the original name. The sql function gives me connection error. 

In [14]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [15]:
!createdb -U dbuser foodinspdb

Creating a new database to populate

In [16]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [17]:
%sql postgresql://dbuser@localhost:5432/foodinspdb

'Connected: dbuser@foodinspdb'

Accessing the database

In [18]:
!csvsql --db postgresql://dbuser@localhost:5432/foodinspdb --insert inspection2010.csv

This populates the newly created database with the inspection csv files. 

In [19]:
%%sql
SELECT COUNT(*) FROM inspection2010

1 rows affected.


count
17556


In [20]:
!csvstat --count inspection2010.csv

Row count: 17556


This verifies that the file has successfully loaded onto the database. We can see that both sql and csvtoolkit return the same value of 17556 for the rows.  

## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

In [22]:
!csvcut -n inspection2010.csv

  1: FACILITY_NAME
  2: FACILITY_KEY
  3: SITE_ADDRESS
  4: CITY
  5: STATE
  6: ZIP
  7: OWNER_KEY
  8: OWNER_NAME
  9: NATURE_OF_BUSINESS
 10: STREET_NUMBER
 11: STREET_NAME
 12: STREET_TYPE
 13: POST_DIRECTIONAL
 14: PHONE
 15: PROGRAM_CATEGORY
 16: PROGRAM_CATEGORY_DESCRIPTION
 17: INSPECTION_DATE
 18: INSPECTION_TYPE
 19: INSPECTION_DESC
 20: SERIAL_NUM
 21: ACTION_CODE
 22: ACTION_DESC
 23: RESULT_CODE
 24: RESULT_DESC
 25: VIOLATION_CODE
 26: VIOLATION_DESC
 27: INSPECTION_MEMO


In [33]:
%%sql
SELECT "RESULT_DESC", COUNT ("FACILITY_NAME")
FROM inspection2010
GROUP BY "RESULT_DESC";

19 rows affected.


RESULT_DESC,count
NOTICE,3
DOWNGRADE,214
PERMIT SUSPENSION,2
EMBARGO,3
RECALL VERIFICATION,16
TRAINING RECEIVED,104
FACILITY CLOSED AT TIME OF INSPECTION,19
OUT OF BUSINESS,142
IN COMPLIANCE,14389
PERMISSION TO OPERATE,1


In [46]:
%%sql
SELECT "VIOLATION_DESC", COUNT ("FACILITY_NAME")
FROM inspection2010
GROUP BY "VIOLATION_DESC";

71 rows affected.


VIOLATION_DESC,count
None,1009
Food improperly labeled,134
Potentially hazardous food improperly thawed,63
Contaminated equipment,20
Hot Holding,175
Improper storage/handling of clean equipment/utens,93
Additional Comments,6353
Plumbing: Improperly installed/maintained/supplied,492
Other Operations - Linen,12
Food contact surfaces,251


That's way too many violation. I would be very weary of eating out if I am ever in Albequerque. 

Now, not all the violations are major. Some are minor, like 'toilet fixture', which does not pose any threat.

In [44]:
%%sql
SELECT DISTINCT "FACILITY_NAME"
FROM inspection2010
WHERE "RESULT_DESC" LIKE '%CLOSURE%';

7 rows affected.


FACILITY_NAME
GRAHAM CENTRAL STATION
VILLA PIZZA INC
CHICHARRONERIA DON CHOCHE
99 BANH SUPERMARKET
OOB ROSYS
OOB MARKET FRESH CAFE
OOB PERICOS ACAPULCO


Here, we identified the restaurants which have been marked to be closed immediately. No we know what to avoid if we are in Albequereue. 

In [53]:
%%sql
SELECT DISTINCT "FACILITY_NAME"
FROM inspection2010
WHERE "VIOLATION_DESC" = 'IMMEDIATE CLOSURE - IMMINENT HEALTH HAZARD'

6 rows affected.


FACILITY_NAME
VILLA PIZZA INC
CHICHARRONERIA DON CHOCHE
99 BANH SUPERMARKET
OOB ROSYS
OOB MARKET FRESH CAFE
OOB PERICOS ACAPULCO


If you notice these are the same restaurants we found as above, where the result desicion was closure. These pose imminnent danger to the public and should be avoided. 

In [59]:
%%sql
SELECT COUNT("VIOLATION_DESC")
FROM inspection2010;

1 rows affected.


count
16547


Here we can see that out of the 17556 rows, 16547 have some violation. Out of which, 3434 have no violations. This brings us to (16547-3434)/17556 = 0.746 or ~75% of restaurants in Albequerque had some violations agaisnt them in FY2010. It is best to avoid eating out at these places if you are ever in the area. 

Also, it looks like all unique facilitties have corresponding result and violation, which was my initial concern. 

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table